# Hero Demo - Wind Shear

In [1]:
%matplotlib inline
import intake
import iris
import datetime
import matplotlib.pyplot as plt
import iris.quickplot as qplt
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import cartopy.feature as cf
hv.extension('bokeh')

/opt/conda/lib/python3.6/site-packages/intake/source/discovery.py:46: UserWarning: Plugin name collision for "netcdf" from
    /opt/conda/lib/python3.6/site-packages/intake_iris/netcdf.py
and
    /opt/conda/lib/python3.6/site-packages/intake_xarray/netcdf.py
Keeping plugin from first location.
  % (plugin_name, orig_path, new_path))


ModuleNotFoundError: No module named 'geoviews'

## Guidence on wind shear for aviation in the UK
We're expecting high shear conditions over the UK this week. This analysis will identify risk regions and issue shear warning accoringly.

In [6]:
direction = intake.cat.mo_aws_earth.mogreps_g.wind_from_direction_at_pressure.read()
speed = intake.cat.mo_aws_earth.mogreps_g.wind_speed_at_pressure.read()

[<iris 'Cube' of wind_from_direction / (degrees) (forecast_period: 67; forecast_reference_time: 14; realization: 18; pressure: 33; latitude: 960; longitude: 1280)>,
 <iris 'Cube' of wind_speed / (m s-1) (forecast_period: 67; forecast_reference_time: 14; realization: 18; pressure: 33; latitude: 960; longitude: 1280)>]

In [8]:
direction

<iris 'Cube' of wind_from_direction / (degrees) (forecast_period: 67; forecast_reference_time: 14; realization: 18; pressure: 33; latitude: 960; longitude: 1280)>

In [ ]:
# ref_time = datetime.datetime(2019, 4, 22, 15, 0)
# pressure_constraint = iris.Constraint(pressure=92500, 
#                                       forecast_period=0,
#                                       forecast_reference_time=ref_time)
# wind_dir = henry_wind_dir.extract(pressure_constraint)
# wind_speed = henry_winf_speed.extract(pressure_constraint)

In [40]:
def calculate_shear(wind_speed, wind_dir):
    # Calculate the cross-product of wind vectors between consecutive pressure levels
    import numpy as np
    import iris
    
    # make copies of cubes
    wind_speed = wind_speed.copy()
    wind_dir = wind_dir.copy()
    
    # remove pressure coordinates in order to out cube difference
    pressure_lvls = wind_speed.coord('pressure')
    wind_speed.remove_coord('pressure')
    wind_dir.remove_coord('pressure')
    
    # work out the parameters for the 
    speed_lower = wind_speed[:, :, :, :, :-1, ...]
    speed_upper = wind_speed[:, :, :, :, 1:, ...]
    dir_diff = wind_dir[:, :, :-1, ...] - wind_dir[:, :, 1:, ...]
    sin_dir_diff = iris.analysis.maths.apply_ufunc(np.sin, dir_diff)
    
    # calculate the cross-product
    shear = speed_lower * speed_upper * sin_dir_diff
    
    # add pressure coord back
    shear.add_dim_coord(dim_coord=pressure_lvls[1:], data_dim=2)
    
    # rename cube and add units
    shear.long_name = "Wind Shear"
    shear.units = "m2 s-2"
    
    return shear

In [41]:
calculate_shear(speed, direction)

ValueError: This operation cannot be performed as there are differing coordinates (realization) remaining which cannot be ignored.

In [ ]:
# ask henry for a cluster

In [ ]:
# pass subsetted datasets to wind shear calculation

shear = calculate_shear(wind_speed, wind_dir)
shear

## Create an annotatable plot

In [ ]:
import holoviews as hv
import geoviews as gv
hv.extension('bokeh')

In [ ]:
# convert cube to gv.Dataset
# OR
# def interactive_contourf()
# ???

def interactive_contourf(cube):
    # Return an interactive GeoViews FilledContours object
    dataset = gv.Dataset(cube, [coord.name() for coord in cube.dim_coords])
    contourf = gv.FilledContours(dataset).opts(line_color=None, line_width=0)
    return contourf

In [ ]:
# create plot and coastlines

shear_plot = interactive_controuf(shear)
coastlines = gv.feature.coastline

In [ ]:
# create warning annotation tools

from holoviews.streams import FreehandDraw

warning_orange = gv.Polygons([]).opts(line_color='orange', line_width=9, 
                                      fill_color='orange', fill_alpha=0.6)
warning_orange_tool = FreehandDraw(source=warning_orange)

In [ ]:
# combine layers into single plot

shear_interactive = shear_plot * coastlines * warning_orange
shear_interactive

## Annotate plot

In [ ]:
# define function to serve up as app

def make_annotable(plot, port=0, websocket_origin='pangeo.informaticslab.co.uk', url_path='annotable'):
    import holoviews as hv
    from bokeh.server.server import Server
    import os
    from IPython.core.display import display, HTML
    import ipywidgets as widgets
    import qrcode
    
    renderer = hv.renderer('bokeh')
    app = renderer.app(plot)
    server = Server({f'/{url_path}': app}, port=port, allow_websocket_origin=[websocket_origin])
    
    server.start()
    
    prefix = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    url = f"https://{websocket_origin}{prefix}proxy/{server.port}/{url_path}"
    display(HTML(f'<a href={url}>{url}</a>'))
    display(qrcode.make(url))
    
    stop_button = widgets.Button(description=f"Stop {url_path}")
    stop_button.on_click(lambda b: server.stop())
    display(stop_button)
    
    return server

In [ ]:
# annotate on phone/ipad

make_annotable(shear_interactive)

In [ ]:
# access annotations directly in notebook

warning_orange_tool.element()

In [ ]:
warning_orange_tool.element().data

## Publish warning as document/website

In [ ]:
# ???